# Introducing new rates in ratings

In [6]:
import numpy as np
import pandas as pd
import pymongo
import bson
from tqdm import tqdm

In [103]:
class MongoServer():
    credentials = None
    conn = None
    db = None
    collections = {}
    def __init__(self, run = False, credentials = "credentials.txt", db_name = "agile_data_science_group_3"):
        if run:
            if not self.connect2Mongo(credentials):
                print("Connection to server Failed.")
            
            if not self.connect2DataBase(db_name):
                print("Connection Data Base Failed.")
    """Rotine to connect to Mongo DB"""    
    def connect2Mongo(self, credentials = "credentials.txt"):
        try:
            #use your database name, user and password here:
            with open(credentials, 'r', encoding='utf-8') as f:
                [name,password,url,dbname]=f.read().splitlines()
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            self.conn = conn
            return True
        except pymongo.errors.ConnectionFailure as e:
            return False
    
    """Routine to connect to a Data Base"""
    def connect2DataBase(self, db_name = "agile_data_science_group_3"):
        try:
            self.db = self.conn[db_name]
            return True
        except:
            return False
    
    """Return the available collections in a data base"""
    def listOfCollections (self):
        return self.db.collection_names()
    
    """Donwload all the ollections from the data base"""
    def getAllCollections (self):
        collections = self.listOfCollections()
        for col in collections:
            self.getCollectionFromServer(col)
        return True
    
    
    """Routine to get one collection of the Data Base"""
    # return the collection
    def getCollectionFromServer (self, name_collection):
        if name_collection in self.db.collection_names():
            self.collections[name_collection] = self.db.get_collection(name_collection)
            print ("Collection ",name_collection," Update in Local.")
            return True
        return False
    
    """ Take the collection from the local copy"""
    def getCollection(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return self.collections[name_collection]
    
    """ Gets all the items of the collection"""
    def getItems(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return [element for element in self.collections[name_collection].find()]
    
    """Query in one Collection"""
    def searchInCollection(self, name_collection, field, patro):
        return [element for element in self.collections[name_collection].find({field:patro})]
    
    """Query in all the Collections"""
    def searchInDB (self, field, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
        return query
    
    """Find one in the collection"""
    def findOne(self, collection_name):
        return self.db.get_collection(collection_name).find_one()
    
    """Froce Brute search"""
    def searchInDBForceBrute(self, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = []
            
            self.searchInCollection(collection, )
            print(self.getItems(collection))
            for element in self.getItems(collection)[0]:
                for key in element.keys():
                    query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
                    
    def findNElement(self, collection_name, N):
        query = []
        for item in self.db.get_collection(collection_name).find().limit( N ):
            query.append(item)
        return query
    
    def insertInCollection (self, collection_name,  item):
        assert type(item) == dict, "Item must be a dictionary"
        return self.db.get_collection(collection_name).insert_one(item)
        
                
        
MS = MongoServer(True)   


In [7]:
ing_1 = pd.read_csv('Source1Ingredients.csv')
ing_2 = pd.read_csv('Source2Ingredients.csv')

ids_1 = np.unique(ing_1['id'])
ids_2 = np.unique(ing_2['id'])

ids = np.concatenate((ids_1, ids_2), axis = 0)

ratings = {}
for _id in tqdm(ids):
    ratings[_id] = np.random.randint(3,5)

100%|████████████████████████████████████████████████████████████████████████| 15560/15560 [00:00<00:00, 265977.25it/s]


In [73]:
colUsers = MS.getItems('users')
__user = []
for _user in colUsers:
    __user.append(_user['_id'])
type(__user)

list

In [57]:
#colRecepie = MS.getItems('recipes')
recipes = []
for _id in tqdm(ids):
    recipes.append(MS.searchInCollection(name_collection='recipes', field='id', patro=_id)[0]['_id'])

100%|████████████████████████████████████████████████████████████████████████████| 15560/15560 [18:47<00:00, 13.80it/s]


In [66]:
len(recipes)

15560

In [143]:
random = np.full((len(ids), len(__user)), np.nan)
for i in tqdm(range(len(ids))):
    for j in range(len(__user)):
        if np.random.randint(0,11) > 9:
            random[i,j] = min(max(int(np.random.normal(ratings[ids[i]], min(1, 5 - ratings[ids[i]]))), 1), 5)
            
np.unique(np.isnan(random), return_counts=True)

100%|██████████████████████████████████████████████████████████████████████████| 15560/15560 [00:02<00:00, 5788.66it/s]


(array([False,  True], dtype=bool), array([ 93261, 933699], dtype=int64))

In [144]:
matrix = pd.DataFrame(random, index=recipes, columns=__user)
matrix

,5a1aa2cb2cfaa80d6ff251f3,5a1aa2cb2cfaa80d6ff251f4,5a1aa2cb2cfaa80d6ff251f5,5a1aa2cb2cfaa80d6ff251f6,5a1aa2cb2cfaa80d6ff251f7,5a1aa2cb2cfaa80d6ff251f8,5a1aa2cb2cfaa80d6ff251f9,5a1aa2cb2cfaa80d6ff251fa,5a1aa2cb2cfaa80d6ff251fb,5a1aa2cb2cfaa80d6ff251fc,...,5a2c5c1394f1473fc6bc3acf,5a2c5c7394f1473fd089e5ad,5a2c5ce594f1473fddb7e56b,5a2c628194f147409a5a0d35,5a2c62ab94f14740a49da76a,5a2c631d94f14740bf1e5d3a,5a2c663094f1474116393736,5a2da41094f1474c8218dbc7,5a2da47194f1474c8218dbc8,5a2f1a7a94f147649a475bac
59fb0d77eee3642f3c0683f7,NaN,NaN,NaN,NaN,1.0,2.0,NaN,3.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59fb0d77eee3642f3c0684a4,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59fb0d77eee3642f3c068640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59fb0d77eee3642f3c068703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN
59fb0d77eee3642f3c068783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59fb0d77eee3642f3c068798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59fb0d77eee3642f3c0687de,NaN,4.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59fb0d77eee3642f3c068861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59fb0d77eee3642f3c068a5a,NaN,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
59fb0d77eee3642f3c068aa0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN


# Insert new data in collection

In [145]:
for user in tqdm(matrix.columns):
    for recepie in matrix.index:
        if not np.isnan(matrix[user][recepie]):
            MS.insertInCollection('ratings', {'user_id': user, 'recipe_id': recepie, 'rating':int(matrix[user][recepie])})

100%|████████████████████████████████████████████████████████████████████████████████| 66/66 [1:35:57<00:00, 87.23s/it]
